**Tutorial 2**

**Before running this code:** Make sure you have the following Julia packages installed: `InteractiveIsing`, `Plots`, `GLMakie`, `FileIO`.

**Install via Julia's package manager**

```julia
using Pkg
Pkg.add(url="https://github.com/rug-minds/InteractiveIsing.jl")
Pkg.add("Plots")
Pkg.add("GLMakie")
Pkg.add("FileIO")
Pkg.add("CairoMakie")
```

**What this tutorial covers:**
This tutorial demonstrates advanced features of InteractiveIsing, including:
- Custom weight functions for different coupling patterns
- Spin glass systems with random and structured disorder

**Prerequisites:**
Please ensure you have completed Tutorial 1 to understand the basic concepts of Ising simulations and hysteresis loops.

In [7]:
## Utility functions for experiments

function newmakie(makietype, args...)
    f = makietype(args...)
    scr = GLMakie.Screen()
    display(scr, f)
    f
end
# Random partition of n into 'parts' segments (returns cumulative sizes)
function random_partition(n, parts)
    if parts == 0
        return [n]
    end
    cuts = sort(rand(1:n-1, parts-1))
    partition = [cuts[1]; diff(cuts); n - cuts[end]]
    return cumsum(partition)
end

# Uniform partition of n into 'parts' nearly equal segments (returns cumulative sizes)
function uniform_partition(n::Integer, parts::Integer)
    if parts == 0
        return [n]
    end
    base = div(n, parts)       # base size of each part
    rem = mod(n, parts)        # remainder to distribute
    partition = [i <= rem ? base + 1 : base for i in 1:parts]
    return cumsum(partition)
end

# Generate random numbers around x with range ±y
function randAround(x::Number, y::Number, dims::Int...)
    x = float(x)
    y = float(y)
    return x .+ y .* (2 .* rand(dims...) .- 1)
end


##################################################################################
### Pulse type: TrianglePulseA (simple four-segment triangular waveform)
struct TrianglePulseA end

function Processes.prepare(::TrianglePulseA, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    first  = LinRange(0, amp, round(Int,rise_point))
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    fourth = LinRange(-amp, 0, round(Int,rise_point))
    pulse = vcat(first, second, third, fourth)
    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        "Wrong length"
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::TrianglePulseA)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


##################################################################################
### Pulse type: PUNDa (no delay time between pulses)
struct PUNDa end

function Processes.prepare(::PUNDa, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    first  = LinRange(0, amp, round(Int,rise_point))
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    fourth = LinRange(-amp, 0, round(Int,rise_point))
    pulse = vcat(first, second, first, second, third, fourth, third, fourth)
    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        println("Pulse is longer than lifetime")
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    all_Es = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::PUNDa)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


##################################################################################
### Pulse type: PUNDb (with delay time between each pulse)
struct PUNDb end

function Processes.prepare(::PUNDb, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    delay  = zeros(Int, rise_point)
    first  = LinRange(0, amp, round(Int,rise_point))
    amp1   = fill(amp, rise_point)
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    amp2   = fill(-amp, rise_point)
    fourth = LinRange(-amp, 0, round(Int,rise_point))

    pulse = vcat(delay, first, amp1, second,
                 delay, first, amp1, second,
                 delay, third, amp2, fourth,
                 delay, third, amp2, fourth)

    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        println("Pulse is longer than lifetime")
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    all_Es = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::PUNDb)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


In [2]:
# Setup: Load packages and create basic 3D Ising graph
# This follows the same pattern as Tutorial 1

using InteractiveIsing, Plots, GLMakie, FileIO
using InteractiveIsing.Processes
import InteractiveIsing as II

# Initialize a 3D Ising graph with specified dimensions and periodic boundary conditions
xL = 100  # Length in the x-dimension
yL = 100  # Length in the y-dimension
zL = 10   # Length in the z-dimension (thin film geometry)

10

**Custom Weight Functions Overview**

Weight functions determine the coupling strength between neighboring spins in the Ising model. They are the core mechanism for defining different interaction patterns and building the adjacency matrix.

**How Weight Functions Work:**

The `@WG` macro creates a `WeightGenerator` that calls your weight function for each pair of dipoles. The process is:

1. **For each dipole at position `(x, y, z)`**:
   - Find all neighbors within `NN` range: `(x+dx, y+dy, z+dz)`
   - Call your weight function with parameters: `(dx, dy, dz, x, y, z, ...)`
   - Store the **returned coupling strength** in the adjacency matrix

2. **The weight function returns the actual coupling coefficient `J_ij`**:
   - This value goes directly into the Hamiltonian: `H = -∑ J_ij s_i s_j`
   - Positive values → ferromagnetic coupling (parallel spins favored)
   - Negative values → antiferromagnetic coupling (anti-parallel spins favored)

**Understanding Distance Scaling with Coefficients:**

```julia
dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
return prefactor / dr2  # This J_ij goes into the Hamiltonian
```

**Important Note on Scaling Approach:**
- **`(dx, dy, dz)`**: Relative displacement in lattice units (e.g., dx=1 means "one lattice step right")
- **`(rx, ry, rz)`**: **User-defined scaling coefficients** - you control these!
- **`dr2`**: Your custom distance metric for determining coupling strength

**Key Point: We Use Scaling Coefficients, Not Physical Units**

In this tutorial, we use **dimensionless scaling coefficients** rather than physical lattice constants. The scaling factors `(rx, ry, rz)` let you:
- **Control relative coupling strengths** between different directions
- **Create anisotropic interactions** (stronger along x vs y vs z)
- **Design interaction patterns** without worrying about physical units
- **Focus on the physics** of spin coupling rather than unit conversion

You can use any distance function you want: `1/dr2`, `1/dr2^1.5`, `exp(-dr2)`, or completely custom rules. The weight function returns the final coupling strength that determines how strongly each pair of dipoles interacts.

In [8]:
# Weight function variant 1: Pure ferromagnetic coupling
function weightfunc1(dr,c1,c2)
    prefac = 1
    d = delta(c1,c2)
    dx, dy, _ = d
    # Always positive coupling (ferromagnetic)
    return prefac / norm2(d)
end

# Weight function variant 2: Checkerboard antiferromagnetic pattern
function weightfunc2(dx, dy, dz, rx, ry, rz)
    # Alternating signs based on lattice position parity
    dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
    
    # Checkerboard pattern: alternating ferro/antiferro based on coordinate sum
    if (abs(dx) + abs(dy)) % 2 == 0
        return 1.0 / dr2    # Ferromagnetic coupling
    else
        return -1.0 / dr2   # Antiferromagnetic coupling
    end
end

weightfunc2 (generic function with 1 method)

**Advanced Application: Spin Glass Systems with Domain-Based Coupling**

Now we'll create complex interaction patterns that produce spin glass behavior using **spatially-varying coupling parameters**. This approach creates realistic material disorder.

**The Strategy: Domain-Based Heterogeneity**

Instead of uniform coupling everywhere, we:

1. **Divide the lattice into domains** using partition functions:
   ```julia
   x_n = random_partition(xL, 20)  # 20 random domains along x
   y_n = uniform_partition(yL, 10) # 10 uniform domains along y
   ```

2. **Assign domain-specific scaling coefficients**:
   ```julia
   coef_x = randAround(1, 0.2, 20)  # Random scaling around 1.0 ± 0.2
   coef_y = randAround(1, 0.2, 10)  # Different for each domain
   ```

3. **Use absolute coordinates `(x, y, z)` to determine which domain contains each dipole**:
   ```julia
   ix = findfirst(v -> x ≤ v, x_n)  # Which x-domain?
   iy = findfirst(v -> y ≤ v, y_n)  # Which y-domain?
   ```

4. **Apply domain-specific scaling to the distance**:
   ```julia
   dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2  # Domain-dependent distance
   ```

**Example Implementation: `weightfunc_glass_anti()`**

```julia
function weightfunc_glass_anti(dx, dy, dz, x, y, z, x_n, y_n, z_n, coef_x, coef_y, coef_z)
    # Step 1: Determine which domain this dipole belongs to
    ix = findfirst(v -> x ≤ v, x_n)  # Domain index along x
    iy = findfirst(v -> y ≤ v, y_n)  # Domain index along y  
    iz = findfirst(v -> z ≤ v, z_n)  # Domain index along z
    
    # Step 2: Use domain-specific scaling coefficients
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 + (coef_z[iz] * dz)^2
    
    # Step 3: Apply interaction pattern (checkerboard antiferromagnetic)
    prefac = (abs(dx) + abs(dy)) % 2 != 0 ? -1 : 1
    
    # Step 4: Return the coupling strength J_ij
    return prefac / dr2
end
```

**What This Achieves:**

1. **Heterogeneous Coupling Strengths**: 
   - Domain A: `coef_x = 1.2` → weaker interactions (`J ∝ 1/1.44`)
   - Domain B: `coef_x = 0.8` → stronger interactions (`J ∝ 1/0.64`)

2. **Realistic Material Disorder**:
   - **Strain/stress variations**: Some regions compressed (`coef < 1`), others stretched (`coef > 1`)
   - **Chemical inhomogeneity**: Different local compositions change interaction strength
   - **Defects and interfaces**: Domain boundaries create frustration and pinning sites

3. **Controlled Frustration** - Systematic rather than purely random disorder

**The Result: Complex Spin Glass Behavior**

When you combine spatial disorder + competing interactions + long-range connections, you get **frustrated energy landscapes** with multiple local energy minima and complex, history-dependent switching behavior.

In [ ]:
x_n = random_partition(xL, 20)
y_n = random_partition(yL, 20)
z_n = random_partition(zL, 3)
coef_x =randAround(1, 0.2, 20)
coef_y =randAround(1, 0.2, 20)
coef_z =randAround(1, 0.2, 3)
println(x_n)
println(y_n)
println(z_n)
println(coef_x)
println(coef_y)
println(coef_z)


function weightfunc_glass_anti(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x,coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    if (abs(dx) + abs(dy)) % 2 != 0  
        prefac = -1
    else  
        prefac = 1
    end
    return prefac / dr2
end

function weightfunc_glass_ferro(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x,coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    prefac = 1
    return prefac / dr2
end

function weightfunc_AntiWithFerro(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x, coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    if ix % 2 == 0 && iy %2==0
        prefac=1
    elseif xor(isodd(ix), isodd(iy))
        if (abs(dx) + abs(dy)) % 2 != 0  
            prefac = -1
        else  
            prefac = 1
        end
    end
    return prefac / dr2
end

wg_glass1 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_glass_ferro(dx,dy,dz,x,y,z,x_n, y_n, z_n, coef_x,coef_y,coef_z)" NN = (2,2,2)
wg_glass2 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_glass_anti(dx,dy,dz,x,y,z, x_n, y_n, z_n, coef_x,coef_y,coef_z)" NN = (2,2,2)
wg_glass3 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_AntiWithFerro(dx, dy, dz, x, y, z, x_n, y_n, z_n, coef_x, coef_y, coef_z)" NN = (2,2,2)


**Advanced Application: Crystal Structure Modeling**

Beyond spin glass systems, weight functions can model **real ferroelectric crystal structures** by incorporating actual crystallographic parameters. This bridges atomistic structure with mesoscopic behavior.

**Crystal Structure Examples:**

Different crystal systems have distinct lattice parameters and symmetries that directly affect ferroelectric properties:

1. **Cubic Crystals** (e.g., BaTiO₃ high-temperature phase):
   - Equal lattice constants: a = b = c
   - Isotropic interactions in all directions
   - No spontaneous polarization (paraelectric)

2. **Tetragonal Crystals** (e.g., PbTiO₃, BaTiO₃ room temperature):
   - a = b ≠ c (elongated or compressed along one axis)
   - Enhanced coupling along the unique axis
   - Spontaneous polarization along c-axis

3. **Orthorhombic Crystals** (e.g., distorted perovskites):
   - a ≠ b ≠ c (three different lattice constants)
   - Complex anisotropic behavior
   - Multiple possible polarization directions

4. **Layered Structures** (e.g., Aurivillius phases):
   - Strong in-plane coupling
   - Weak interlayer interactions
   - Quasi-2D ferroelectric behavior

**Key Advantages:**
- **Quantitative modeling** using experimental lattice constants
- **Temperature-dependent** structural transitions
- **Structure-property** relationship investigations
- **Material comparison** and design

In [ ]:
# ========================================================================
# CRYSTAL STRUCTURE WEIGHT FUNCTIONS
# ========================================================================

# Cubic crystal structure (e.g., BaTiO₃ cubic phase)
function cubic_weightfunc(dx, dy, dz, a = 4.0)
    """
    Cubic crystal: a = b = c, α = β = γ = 90°
    Example: BaTiO₃ cubic phase (a ≈ 4.0 Å)
    """
    dr2 = (a*dx)^2 + (a*dy)^2 + (a*dz)^2
    return 1.0 / dr2
end

# Tetragonal crystal structure (e.g., PbTiO₃)
function tetragonal_weightfunc(dx, dy, dz, a = 3.9, c = 4.1)
    """
    Tetragonal crystal: a = b ≠ c, α = β = γ = 90°
    Example: PbTiO₃ (a ≈ 3.9 Å, c ≈ 4.1 Å)
    c/a ≈ 1.05 indicates elongation along z-axis
    """
    dr2 = (a*dx)^2 + (a*dy)^2 + (c*dz)^2
    return 1.0 / dr2
end

# Orthorhombic crystal structure (e.g., GdFeO₃-type)
function orthorhombic_weightfunc(dx, dy, dz, a = 3.8, b = 3.9, c = 4.0)
    """
    Orthorhombic crystal: a ≠ b ≠ c, α = β = γ = 90°
    Example: Distorted perovskite structures
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    dr2 = (a*dx)^2 + (b*dy)^2 + (c*dz)^2
    return 1.0 / dr2
end

# Layered structure with reduced interlayer coupling
function layered_weightfunc(dx, dy, dz, a = 3.9, c = 25.0, λ = 0.1)
    """
    Layered structure with weak interlayer coupling
    Example: Aurivillius phases, Dion-Jacobson phases
    λ = interlayer coupling reduction factor
    """
    dr2 = (a*dx)^2 + (a*dy)^2 + (c*dz)^2
    
    # Reduce coupling for interlayer interactions (dz ≠ 0)
    interlayer_factor = (dz == 0) ? 1.0 : λ
    return interlayer_factor / dr2
end

# Example weight generators for different crystal structures
println("Creating weight generators for crystal structures...")

# Cubic BaTiO₃-like
wg_cubic = @WG "(dx,dy,dz,x,y,z) -> cubic_weightfunc(dx,dy,dz,4.0)" NN = (2,2,2)

# Tetragonal PbTiO₃-like  
wg_tetragonal = @WG "(dx,dy,dz,x,y,z) -> tetragonal_weightfunc(dx,dy,dz,3.9,4.1)" NN = (2,2,2)

# Orthorhombic distorted perovskite
wg_orthorhombic = @WG "(dx,dy,dz,x,y,z) -> orthorhombic_weightfunc(dx,dy,dz,3.8,3.9,4.0)" NN = (2,2,2)

# Layered structure
wg_layered = @WG "(dx,dy,dz,x,y,z) -> layered_weightfunc(dx,dy,dz,3.9,25.0,0.1)" NN = (2,2,2)

println("Crystal structure weight generators created!")
println("Usage: genAdj!(g[1], wg_tetragonal)  # Use any of: wg_cubic, wg_tetragonal, wg_orthorhombic, wg_layered")

**Research Applications with Crystal Structure Models:**

 

These crystal structure weight functions enable **quantitative ferroelectric research**:

 

🔬 **Experimental Validation**: 

- Use X-ray diffraction data → lattice parameters → simulation input

- Compare simulated vs experimental domain patterns

 

📊 **Structure-Property Studies**:

- How does c/a ratio affect coercive field in tetragonal crystals?

- What switching mechanisms occur in orthorhombic systems?

- How do layered structures influence polarization fatigue?

 

🌡️ **Phase Transition Modeling**:

- Cubic ↔ Tetragonal transitions (temperature-dependent parameters)

- Structural evolution during field cycling

- Thermal expansion effects

 

💡 **Material Design**:

- Optimize lattice parameters for specific applications

- Predict properties of new compositions

- Guide synthesis of engineered structures

 

---

 

⚠️ **Important note: limitations of the dr2 scaling model**

 

A simple 1/dr² distance decay is **not** universally appropriate.

 

**Physical considerations:**

1. Real dipole–dipole interaction scales as ~1/r³; using 1/r² is an approximation

2. Exchange interactions often decay exponentially ~e^(−r/a) (strongly correlated systems)

3. Electrostatic screening can be significant in high‑κ materials

4. Quantum tunneling may matter in thin films or nanostructures

 

**When to modify the distance law:**

- Strongly correlated oxides: consider exponential decay, e.g., exp(−√dr2/λ)

- Near metallic gates/electrodes: include image‑charge corrections

- Surfaces and interfaces: broken symmetry induces anisotropy

- Quantum dot arrays: tunneling‑dominated coupling

 

**Suggested alternative distance laws (sketches):**

```julia

# Exponential decay (strongly correlated systems)

return J₀ * exp(-sqrt(dr2)/λ)

 

# Modified dipolar interaction

return J₀ / (dr2^1.5)  # closer to 1/r³

 

# Screened Coulomb interaction

return J₀ * exp(-sqrt(dr2)/λ_screen) / dr2

 

# Anisotropy for layered materials

aniso_factor = (dz == 0) ? 1.0 : exp(-abs(dz)/d_layer)

return J₀ * aniso_factor / dr2

```

 

**Physical sanity checklist when defining weight functions:**

✅ Does the distance decay follow known physics?  

✅ Is the anisotropy consistent with crystal symmetry?  

✅ Are boundary conditions reasonable?  

✅ Numerically stable (avoid divergences)?

 

---

 

**🚀 Suggestions: what else to explore?**

 

1. **Multiscale modeling** 🔬

- Atomistic → mesoscopic: from DFT to continuum models

- Time scales: femtosecond pulses → quasi‑static switching

- Spatial scales: nanoscale domain walls → device scale (mm)

 

2. **Machine‑learning augmentation** 🤖

- Automatic parameter tuning: fit weight functions to experiments

- Phase‑transition prediction: infer Tc from structural parameters

- Inverse design: given performance targets, design optimal structures

 

3. **Non‑equilibrium dynamics** ⚡

- Ultrafast switching: domain evolution under femtosecond pulses

- Fatigue mechanisms: micro‑damage accumulation under cycling

- Thermal effects: Joule heating impact on switching

 

4. **Topological ferroelectrics** 🌀

- Domain‑wall topology: vortices, skyrmions

- Topological transitions: continuous deformation of domain textures

- Protected states: topologically robust ferroelectric structures

 

5. **Heterostructure engineering** 🔧

- Interfacial coupling: ferroelectric/magnetic/superconducting multilayers

- Strain engineering: substrate‑induced phases

- Size effects: ferroelectricity under quantum confinement

 

6. **Experiment–theory loop** 🔄

- Real‑time feedback: auto‑update model parameters from data

- Predict→verify cycle: theory → experiment → model refinement

- Digital twins: virtual replicas of real devices

 

**Most promising directions (my take):**

🎯 Multiphysics coupling: electro‑thermal‑mechanical‑optical co‑modeling  

🎯 Materials informatics: high‑throughput workflows + data mining  

🎯 Quantum ferroelectrics: phenomena dominated by quantum effects

 

**Practical next steps:**

1. Choose a specific material (e.g., PbTiO₃)

2. Collect experimental inputs (lattice constants, domain patterns)

3. Implement a custom weight function

4. Compare simulation vs experiment

5. Look for new physical insights!

 

**Next Steps**: Try replacing `wg_glass2` below with `wg_tetragonal` to see how crystal structure affects hysteresis behavior!


In [ ]:
# ========================================================================
# More Physically Realistic Distance Functions Examples
# ========================================================================

# Exponential decay (strongly correlated systems, e.g., transition metal oxides)
function exponential_decay_weightfunc(dx, dy, dz, a = 3.9, λ = 2.0, J₀ = 1.0)
    """
    Exponential decay interaction: J ∝ exp(-r/λ)
    Applicable to: strongly correlated electron systems, exchange interactions
    λ: correlation length, typically a few lattice constants
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    r = sqrt((a*dx)^2 + (a*dy)^2 + (a*dz)^2)
    return J₀ * exp(-r/λ)
end

# Real dipole interaction (1/r³ decay)
function dipole_interaction_weightfunc(dx, dy, dz, a = 3.9, J₀ = 1.0)
    """
    Real dipole-dipole interaction: J ∝ 1/r³
    Closer to real long-range electrostatic interactions
    Note: numerically need to carefully handle nearest-neighbor divergence
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    dr2 = (a*dx)^2 + (a*dy)^2 + (a*dz)^2
    r = sqrt(dr2)
    # Add small cutoff to avoid numerical divergence
    r_cutoff = max(r, 0.5*a)
    return J₀ / (r_cutoff^3)
end

# Screened Coulomb interaction (e.g., Thomas-Fermi screening)
function screened_coulomb_weightfunc(dx, dy, dz, a = 3.9, λ_screen = 5.0, J₀ = 1.0)
    """
    Screened Coulomb interaction: J ∝ exp(-r/λ)/r
    Applicable to: metallic systems, high carrier density materials
    λ_screen: screening length (Debye length)
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    r = sqrt((a*dx)^2 + (a*dy)^2 + (a*dz)^2)
    return J₀ * exp(-r/λ_screen) / r
end

# Anisotropic layered structure
function anisotropic_layered_weightfunc(dx, dy, dz, a = 3.9, c = 25.0, d_layer = 2.0, J₀ = 1.0)
    """
    Anisotropic layered structure
    Intralayer: normal interactions
    Interlayer: exponential decay
    d_layer: interlayer coupling decay length
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    
    # Intralayer interactions (dz = 0)
    if dz == 0
        dr2 = (a*dx)^2 + (a*dy)^2
        return J₀ / (dr2 + (0.5*a)^2)  # Add small regularization
    else
        # Interlayer interactions (dz ≠ 0)
        dr2 = (a*dx)^2 + (a*dy)^2 + (c*dz)^2
        interlayer_decay = exp(-abs(c*dz)/d_layer)
        return J₀ * interlayer_decay / (dr2 + (0.5*a)^2)
    end
end

# Strain-modulated interactions
function strain_modulated_weightfunc(dx, dy, dz, x, y, z, a = 3.9, strain_grad = 0.01, J₀ = 1.0)
    """
    Strain gradient modulated interactions
    Simulate substrate strain, thermal stress effects
    strain_grad: strain gradient strength
    """
    if dx == 0 && dy == 0 && dz == 0
        return 0.0
    end
    
    # Local strain varies with position
    local_strain = 1.0 + strain_grad * (x/100.0)  # normalized position
    effective_a = a * local_strain
    
    dr2 = (effective_a*dx)^2 + (effective_a*dy)^2 + (effective_a*dz)^2
    return J₀ / dr2
end

println("Creating more physically realistic weight functions...")

# Create these advanced weight generators
wg_exponential = @WG "(dx,dy,dz,x,y,z) -> exponential_decay_weightfunc(dx,dy,dz,3.9,2.0,1.0)" NN = (3,3,3)
wg_dipole = @WG "(dx,dy,dz,x,y,z) -> dipole_interaction_weightfunc(dx,dy,dz,3.9,1.0)" NN = (3,3,3)
wg_screened = @WG "(dx,dy,dz,x,y,z) -> screened_coulomb_weightfunc(dx,dy,dz,3.9,5.0,1.0)" NN = (3,3,3)
wg_anisotropic = @WG "(dx,dy,dz,x,y,z) -> anisotropic_layered_weightfunc(dx,dy,dz,3.9,25.0,2.0,1.0)" NN = (3,3,3)
wg_strain = @WG "(dx,dy,dz,x,y,z) -> strain_modulated_weightfunc(dx,dy,dz,x,y,z,3.9,0.01,1.0)" NN = (2,2,2)

println("Advanced weight functions created!")
println("Usage: genAdj!(g[1], wg_exponential)  # or other wg_xxx")
println("Note: Physical parameters of these functions need to be adjusted for specific materials!")

**Simulation Examples**

Now we'll run three different simulations to demonstrate how different weight functions affect the hysteresis behavior:

1. **Example 1**: Standard ferromagnetic coupling with uniform interactions
2. **Example 2**: Spin glass with antiferromagnetic checkerboard pattern  
3. **Example 3**: Mixed ferro/antiferromagnetic domains

Each simulation uses the same external field protocol (triangular pulse) but different internal coupling patterns. Compare the resulting hysteresis loops to see how the microscopic interactions influence macroscopic behavior.

In [ ]:
# Example 1: normal weight function (ferro)
g = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g.hamiltonian = Ising(g) + DepolField(g, c=6000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g.hamiltonian = sethomogenousparam(g.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.

homogeneousself!(g, -1)



### Change nearest neighbours
####################################################################################
wg1 = @WG weightfunc1 NN = (2,2,2)
####################################################################################
genAdj!(g[1], wg1)

### We set Temperature to 1.5
temp(g,4)
fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)
### estimate time
est_remaining(process(g))

# Wait until it is done
args = process(g) |> fetch # If you want to close ctr+c
# args = process(g) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w2=newmakie(lines, voltage, Pr)
# w3=newmakie(lines,Pr)


Layers or WGs: (InteractiveIsing.LayerProperties((100, 100, 10), (stateset = nothing, stype = Continuous(), weights = nothing, periodic = (:x, :y))),)
layers_props = (InteractiveIsing.LayerProperties((100, 100, 10), (stateset = nothing, stype = Continuous(), weights = nothing, periodic = (:x, :y))),)
Making weights for layer Discrete() IsingLayer 1 with size (100, 100, 10) and stateset (-1, 1)

 with connections:
 and 0 defects
Graph: Discrete() IsingLayer 1 with size (100, 100, 10) and stateset (-1, 1)

 with connections:
 and 0 defects
InteractiveIsing.LayoutPanel
BaseInteractiveIsing.LayoutPanel
Base.RefValue{Float32}(0.0f0)
params: Expr[:($(Expr(:kw, :NN, :((2, 2, 2)))))]
Function argnames are: [:dr, :c1, :c2]
Function location is: global
.RefValue{Float32}(0.0f0)
params: Expr[:($(Expr(:kw, :NN, :((2, 2, 2)))))]
Function argnames are: [:dr, :c1, :c2]
Function location is: global
Estimated time to completion: 0:0:56
Of which remaining: 0:0:56
Estimated time to completion: 0:0:56
Of 

In [12]:
g.hamiltonian = Ising(g) + DepolField(g, c=6000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g.hamiltonian = sethomogenousparam(g.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.

homogeneousself!(g, 10)



### Change nearest neighbours
####################################################################################
wg1 = @WG weightfunc1 NN = (2,2,2)
####################################################################################
genAdj!(g[1], wg1)

### We set Temperature to 1.5
temp(g,4)
fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)
### estimate time
est_remaining(process(g))

# Wait until it is done
args = process(g) |> fetch # If you want to close ctr+c
# args = process(g) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w2=newmakie(lines, voltage, Pr)
# w3=newmakie(lines,Pr)

Base.RefValue{Float32}(0.0f0)
params: Expr[:($(Expr(:kw, :NN, :((2, 2, 2)))))]
Function argnames are: [:dr, :c1, :c2]
Function location is: global
Estimated time to completion: 0:1:12
Of which remaining: 0:1:12
Estimated time to completion: 0:1:12
Of which remaining: 0:1:12


In [33]:
dr=1
c1 = coord(1,1,1)
c2 = coord(1,1,2)
weightfunc1(dr, c1, c2)
wg1.func(dr,c1,c2)

UndefVarError: UndefVarError: `weightfunc1` not defined in `InteractiveIsing`
Suggestion: check for spelling errors or missing imports.

**Results Analysis - Example 1 vs Example 2**

Compare the hysteresis loops from the two simulations:

- **Example 1 (ferro)**: Should show clean, symmetric loops with sharp switching
- **Example 2 (spin glass anti)**: Expect more rounded loops, possible pinching, and asymmetric behavior

The spin glass system creates **competing interactions** that lead to:
- Frustrated energy landscapes
- Multiple metastable states  
- History-dependent switching behavior
- Reduced coercive field uniformity

In [ ]:
# Example 2：spin glass anti alignment
g_glass_anti = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g_glass_anti)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g_glass_anti.hamiltonian = Ising(g_glass_anti) + DepolField(g_glass_anti, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g_glass_anti.hamiltonian = sethomogenousparam(g_glass_anti.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.
homogeneousself!(g_glass_anti, -1)

### We set Temperature to 1.5
settemp(g_glass_anti,1.5)


genAdj!(g_glass_anti[1], wg_glass2)

fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g_glass_anti, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)

### estimate time
est_remaining(process(g_glass_anti))

# Wait until it is done
args = process(g_glass_anti) |> fetch # If you want to close ctr+c
# args = process(g_glass_anti) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w_anti_2=newmakie(lines, voltage, Pr)
# w_anti_3=newmakie(lines,Pr)
inlineplot() do 
    lines(voltage, Pr)
end

**Example 3: Mixed Ferro/Antiferromagnetic Domains**

This simulation demonstrates the most complex case where:
- **Even-indexed domains**: Pure ferromagnetic behavior
- **Odd-indexed domains**: Mixed ferro/antiferromagnetic patterns based on XOR logic
- **Domain boundaries**: Create additional frustration and pinning sites

Expected behavior:
- Intermediate loop characteristics between Examples 1 and 2
- Possible multi-step switching due to domain heterogeneity
- Enhanced coercive field distribution

In [ ]:
# Example 3：spin glass with mixed interactions
g_GlassMix = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g_GlassMix)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g_GlassMix.hamiltonian = Ising(g_GlassMix) + DepolField(g_GlassMix, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g_GlassMix.hamiltonian = sethomogenousparam(g_GlassMix.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.
homogeneousself!(g_GlassMix, -1)

### We set Temperature to 1.5
settemp(g_GlassMix,1.5)


genAdj!(g_GlassMix[1], wg_glass3)

fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g_GlassMix, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)

### estimate time
est_remaining(process(g_GlassMix))

# Wait until it is done
args = process(g_GlassMix) |> fetch # If you want to close ctr+c
# args = process(g_GlassMix) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

w_Mix_2=newmakie(lines, voltage, Pr)    
# w_Mix_3=newmakie(lines,Pr)
inlineplot() do 
    lines(voltage, Pr)
end